<a href="https://colab.research.google.com/github/souradip93/CS69002_9A_18CS60R07/blob/master/Assignment9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import the libraries**

In [0]:
import torch
import pandas as pd
import numpy as np
import io
import nltk
import re

# **Upload Train file**

Option 1 - Upload the trainig dataset

In [0]:
from google.colab import files
uploaded = files.upload()
df_train = pd.read_csv(io.StringIO(uploaded['Train_20K.txt'].decode('utf-8')), sep='\t')
df_train.head()


uploaded = files.upload()
df_test = pd.read_csv(io.StringIO(uploaded['Test_5K.txt'].decode('utf-8')), sep='\t')
df_test.head()

*OR*

Option 2 - Store the file in your google drive account

In [100]:
from google.colab import drive
drive.mount('/content/drive')
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Train_20K.txt', sep='\t')
df_train.head()
df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Test_5K.txt', sep='\t')
df_test.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,text,label
0,as always this is an inaccurate picture of the...,0
1,Did the movie-makers even preview this before ...,0
2,"Heavily re-edited and often confusing, the ori...",0
3,I notice that most of the people who think thi...,0
4,"First of all, this is a low-budget movie, so m...",0


In [0]:
train_data_X = df_train['text'].astype(str).tolist()
train_data_Y = df_train['label'].astype(int)

In [0]:
test_data_X = df_test['text'].astype(str).tolist()
test_data_Y = df_test['label'].astype(int)

# **Preprocessing**

1. Remove br tags and numbers

In [0]:
def remove_br_tags_and_numbers(train_data_X):
  import re
  regex = re.compile(r'<br.*?>|\d+')
  train_data_X = [regex.sub(' ', x) for x in train_data_X]
  return train_data_X

2. Tokenize - Consider only alphanumeric characters

In [0]:

def tokenize(train_data_X):
  #from nltk.tokenize import RegexpTokenizer
  #tokenizer = RegexpTokenizer(r'\w+')
  #train_data_X = [tokenizer.tokenize(x) for x in train_data_X]
  train_data_X = [x.split(' ') for x in train_data_X]
  train_data_X = [[y for y in x if y != ''] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X


3. Convert to lower case

In [0]:
def convert_to_lower(train_data_X):
  train_data_X = [[y.lower() for y in x] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X

4. Remove stopwords

In [0]:
def remove_stopwords(train_data_X):
  from nltk.corpus import stopwords
  nltk.download('stopwords')
  stopword_set = set(stopwords.words('english'))
  train_data_X = [[y for y in x if y not in stopword_set] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X

5. Perform lemmatization

In [0]:
def lemmatize(train_data_X):
  from nltk.stem import WordNetLemmatizer 
  nltk.download('wordnet')
  lemmatizer = WordNetLemmatizer()
  train_data_X = [[lemmatizer.lemmatize(y) for y in x] for x in train_data_X]
  return train_data_X

In [0]:
def remove_punctuation(train_data_X):
  import string
  table = str.maketrans(' ', ' ', string.punctuation)
  train_data_X = [x.translate(table) for x in train_data_X]
  return train_data_X

**Preprocessing on Training data**

In [117]:
train_data_X = remove_br_tags_and_numbers(train_data_X)
train_data_X = remove_punctuation(train_data_X)
train_data_X = tokenize(train_data_X)
train_data_X = convert_to_lower(train_data_X)
train_data_X = remove_stopwords(train_data_X)
train_data_X = lemmatize(train_data_X)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Preprocessing on Test data**

In [120]:
test_data_X = remove_br_tags_and_numbers(test_data_X)
test_data_X = remove_punctuation(test_data_X)
test_data_X = tokenize(test_data_X)
test_data_X = convert_to_lower(test_data_X)
test_data_X = remove_stopwords(test_data_X)
test_data_X = lemmatize(test_data_X)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#**Reduce Vocab**

In [0]:
def generate_word_frequency(dataset):
  word_frequency = {}
  for sent in dataset:
      for word in sent:
          if word not in word_frequency:
              word_frequency[word] = 1
          else:
              val = word_frequency[word]
              val += 1
              word_frequency[word] = val
              
  #word_to_ix['UNKNOWN'] = len(word_to_ix)
  return word_frequency

In [0]:
THRESHOLD_FREQUENCY = 10

def reduce_vocab(word_frequency):
  word_to_ix = {}
  for word in word_frequency:
    val = word_frequency[word]
    if val >= THRESHOLD_FREQUENCY:
      word_to_ix[word] = len(word_to_ix)
  word_to_ix['UNKNOWN'] = len(word_to_ix)
  return word_to_ix

In [122]:
word_frequency = generate_word_frequency(train_data_X + test_data_X)
word_to_ix = reduce_vocab(word_frequency)

VOCAB_SIZE = len(word_to_ix)
print(VOCAB_SIZE)

17254


Store the word index once created

In [0]:
import pickle

pickle_out = open("word_to_ix.pickle","wb")
pickle.dump(word_to_ix, pickle_out)
pickle_out.close()

from google.colab import files
model = files.download("word_to_ix.pickle")


Irrelevant for this Task. Remove it later

In [124]:
label_to_ix = {1: 1, 0: 0}
ix_to_label = {v: k for k, v in label_to_ix.items()}

label_to_ix, ix_to_label

({0: 0, 1: 1}, {0: 0, 1: 1})

#**Pytorch initialization**

In [126]:
import torch.nn as nn
import torch.nn.functional as F

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.device('cuda')



device(type='cuda')

#**Create bow vectors**

In [0]:
from torch.autograd import Variable
import time

def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[ word_to_ix['UNKNOWN'] ] += 1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    #print(label)
    return torch.LongTensor([label_to_ix[label]])

bow_vec_orig = []
for instance in train_data_X:
  ins_bow_vector = make_bow_vector(instance, word_to_ix)
  bow_vec_orig.append(Variable(ins_bow_vector).cuda())
  
label_orig = []
for l in train_data_Y:
  label_vector = make_target(l, label_to_ix)
  label_orig.append(Variable(label_vector).cuda())

#**Functions to train model and find metrics**

In [0]:
def train(task, loss_function, opt):
  bow_vec = bow_vec_orig[:]
  label = label_orig[:]
  # the training loop
  for epoch in range(10):
      start = time.time()
      for ind, instance in enumerate(train_data_X):
          task.zero_grad()
          probs = task(bow_vec[ind]) # forward pass
          loss = loss_function(probs, label[ind])
          loss.backward()
          opt.step()
      end = time.time()
      print('Time Taken - ' + str(end - start))
      print('Epoch - ' + str(epoch) + ' , LOSS - ' + str(loss.data))
  return task
      
def calculateMetrics(task):
  print('--- AFTER TRAINING ---')
  tp = 0
  tn = 0
  fp = 0
  fn = 0

  for ind, instance in enumerate(test_data_X):
      bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
      logprobs = task(bow_vec)
      pred = np.argmax(logprobs.cpu().data.numpy())
      label = test_data_Y[ind]
      if label == 1 and ix_to_label[pred] == 1:
        tp += 1
      elif label == 1 and ix_to_label[pred] == 0:
        fn += 1
      elif label == 0 and ix_to_label[pred] == 0:
        tn += 1
      else:
        fp += 1

  print('Accuracy - ' + str(float(tp+tn)/ len(test_data_X)))
  print('Recall - ' + str(float(tp)/ (tp+fn)))
  print('Precision - ' + str(float(tp)/ (tp+fp)))
  
def save_model(task, name):
  import torch
  torch.save(task,name)

  from google.colab import files
  model = files.download(name)

#**Neural network for Task 1**

In [0]:
class Task1(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons, vocab_size):
    super(Task1, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons)
    self.lin2 = nn.Linear(hidden_layer_neurons, num_labels)
    
  def forward(self, x):
    out = self.lin1(x)
    return F.softmax(self.lin2(out))

Setting model parameters for Task 1 A

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS = 50
  
task1a = Task1(NUM_LABELS, HIDDEN_LAYER_NEURONS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function1a = nn.NLLLoss()
opt1a = torch.optim.SGD(task1a.parameters(), lr = 0.1)

Setting model parameters for Task 1 B

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS = 100
  
task1b = Task1(NUM_LABELS, HIDDEN_LAYER_NEURONS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function1b = nn.NLLLoss()
opt1b = torch.optim.SGD(task1b.parameters(), lr = 0.1)

Setting model parameters for Task 1 C

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS = 150
  
task1c = Task1(NUM_LABELS, HIDDEN_LAYER_NEURONS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function1c = nn.NLLLoss()
opt1c = torch.optim.SGD(task1c.parameters(), lr = 0.1)

#**Training for Task 1**

**Task 1 a**

*TRAINING*

In [0]:
task = train(task1a, loss_function1a, opt1a)
calculateMetrics(task)
save_model(task, 'model_1a.json')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


Time Taken - 16.690893411636353
Epoch - 0 , LOSS - tensor(-1., device='cuda:0')


**Task 1 b**

*TRAINING*

In [0]:
task = train(task1b, loss_function1b, opt1b)
calculateMetrics(task)
save_model(task, 'model_1b.json')


**Task 1 c**

*TRAINING*

In [0]:
task = train(task1a, loss_function1c, opt1c)
calculateMetrics(task)
save_model(task, 'model_1c.json')


#**Neural network for Task 2**

In [0]:
class Task2(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons1, hidden_layer_neurons2, vocab_size):
    super(Task2, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons1)
    self.lin2 = nn.Linear(hidden_layer_neurons1, hidden_layer_neurons2)
    self.lin3 = nn.Linear(hidden_layer_neurons2, num_labels)
    
  def forward(self, x):
    out = self.lin1(x)
    return F.softmax(self.lin3(self.lin2(out)))
    

Setting model parameters for Task 2 A

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 10
HIDDEN_LAYER_NEURONS2 = 10
  
task2a = Task2(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function2a = nn.NLLLoss()
opt2a = torch.optim.SGD(task2a.parameters(), lr = 0.1)

Setting model parameters for Task 2 B

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 20
HIDDEN_LAYER_NEURONS2 = 10
  
task2b = Task2(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function2b = nn.NLLLoss()
opt2b = torch.optim.SGD(task2b.parameters(), lr = 0.1)

Setting model parameters for Task 2 C

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 30
HIDDEN_LAYER_NEURONS2 = 30
  
task2c = Task2(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function2c = nn.NLLLoss()
opt2c = torch.optim.SGD(task2c.parameters(), lr = 0.1)

Setting model parameters for Task 2 D

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 50
HIDDEN_LAYER_NEURONS2 = 50
  
task2d = Task2(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function2d = nn.NLLLoss()
opt2d = torch.optim.SGD(task2d.parameters(), lr = 0.1)

Setting model parameters for Task 2 E

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 100
HIDDEN_LAYER_NEURONS2 = 50
  
task2e = Task2(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function2e = nn.NLLLoss()
opt2e = torch.optim.SGD(task2e.parameters(), lr = 0.1)

#**Training for Task 2**

**Task 2 a**

*TRAINING*

In [0]:
task = train(task2a, loss_function2a, opt2a)
calculateMetrics(task)
save_model(task, 'model_2a.json')


**Task 2 b**

*TRAINING*

In [0]:
task = train(task2b, loss_function2b, opt2b)
calculateMetrics(task)
save_model(task, 'model_2b.json')


**Task 2c**

*TRAINING*

In [0]:
task = train(task2c, loss_function2c, opt2c)
calculateMetrics(task)
save_model(task, 'model_2c.json')


**Task 2 d**

*TRAINING*

In [0]:
task = train(task2d, loss_function2d, opt2d)
calculateMetrics(task)
save_model(task, 'model_2d.json')


**Task 2 e**

*TRAINING*

In [0]:
task = train(task1a, loss_function2e, opt2e)
calculateMetrics(task)
save_model(task, 'model_2e.json')


#**Neural network for Task 3**

In [0]:
class Task3(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons1, hidden_layer_neurons2, hidden_layer_neurons3, vocab_size):
    super(Task2, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons1)
    self.lin2 = nn.Linear(hidden_layer_neurons1, hidden_layer_neurons2)
    self.lin3 = nn.Linear(hidden_layer_neurons2, num_labels)
    self.lin4 = nn.Linear(hidden_layer_neurons3, num_labels)
    
  def forward(self, x):
    out = self.lin1(x)
    return F.softmax(self.lin4(self.lin3(self.lin2(out))))

Setting model parameters for Task 3 A

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 100
HIDDEN_LAYER_NEURONS2 = 50
HIDDEN_LAYER_NEURONS3 = 10
  
task3a = Task2(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, HIDDEN_LAYER_NEURONS3, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function3a = nn.NLLLoss()
opt3a = torch.optim.SGD(task3a.parameters(), lr = 0.1)

Setting model parameters for Task 3 B

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 200
HIDDEN_LAYER_NEURONS2 = 100
HIDDEN_LAYER_NEURONS3 = 10
  
task3b = Task2(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, HIDDEN_LAYER_NEURONS3, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function3b = nn.NLLLoss()
opt3b = torch.optim.SGD(task3b.parameters(), lr = 0.1)

#**Training for Task 3**

**Task 3 a**

*TRAINING*

In [0]:
task = train(task3a, loss_function3a, opt3a)
calculateMetrics(task)
save_model(task, 'model_3a.json')


**Task 3 b**

*TRAINING*

In [0]:
task = train(task3b, loss_function3b, opt3b)
calculateMetrics(task)
save_model(task, 'model_3b.json')


In [0]:
from google.colab import files
temp_test = files.upload()


In [0]:
import torch
bow = torch.load(io.BytesIO(temp_test['model.json']))

In [0]:
from google.colab import files
temp_test = files.upload()

In [0]:
pickle_in = open("word_to_ix.pickle","rb")
word_to_ix = pickle.load(pickle_in)